In [1]:
import ee
import geemap
# import geopandas as gpd
# import geemap.colormaps as cm
from ipywidgets import widgets
from IPython.display import display
import datetime

# Authenticate and initialize Google Earth Engine API
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

In [2]:
# Define the region of interest (e.g., Madhya Pradesh or any other state)
# Replace 'MP' with the correct FeatureCollection or boundary of your region
MP = ee.FeatureCollection("FAO/GAUL/2015/level1") \
    .filter(ee.Filter.eq("ADM1_NAME", "Madhya Pradesh"))

## Visualization 



In [10]:
# Create a map
Map = geemap.Map()

# Function to create LST layers
def create_lst_layers(year1, year2):
    def load_lst(year):
        return ee.ImageCollection("MODIS/061/MOD11A1") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .select('LST_Day_1km') \
            .map(lambda img: img.multiply(0.02).subtract(273.15))  # Convert Kelvin to Celsius
    
    try:
        lst1 = load_lst(year1).mean().clip(MP)
        lst2 = load_lst(year2).mean().clip(MP)

        vis_params = {
            'min': 20,
            'max': 40,
            'palette': ['blue', 'green', 'yellow', 'orange', 'red']
        }

        Map.addLayer(lst1, vis_params, f'Mean LST {year1} (°C)')
        Map.addLayer(lst2, vis_params, f'Mean LST {year2} (°C)')

    except Exception as e:
        print(f"Error loading LST layers: {e}")

# Function to create rainfall layers
def create_rainfall_layers(year1, year2):
    def load_rainfall(year):
        return ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .sum()

    try:
        rainfall1 = load_rainfall(year1).clip(MP)
        rainfall2 = load_rainfall(year2).clip(MP)

        vis_params = {
            'min': 500,
            'max': 2000,
            'palette': ['blue', 'lightblue', 'green', 'yellow', 'red']
        }

        Map.addLayer(rainfall1, vis_params, f'Annual Rainfall {year1} (mm)')
        Map.addLayer(rainfall2, vis_params, f'Annual Rainfall {year2} (mm)')

    except Exception as e:
        print(f"Error loading rainfall layers: {e}")

# Function to create humidity layers
def create_humidity_layers(year1, year2):
    def load_humidity(year):
        return ee.ImageCollection("ECMWF/ERA5/MONTHLY") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .select('mean_2m_air_temperature', 'dewpoint_2m_temperature') \
            .mean() \
            .expression(
                '(exp((17.625 * TD)/(243.04 + TD)) / exp((17.625 * T)/(243.04 + T))) * 100',
                {'TD': 'dewpoint_2m_temperature', 'T': 'mean_2m_air_temperature'}
            )

    try:
        humidity1 = load_humidity(year1).clip(MP)
        humidity2 = load_humidity(year2).clip(MP)

        vis_params = {
            'min': 30,
            'max': 90,
            'palette': ['purple', 'blue', 'green', 'yellow', 'red']
        }

        Map.addLayer(humidity1, vis_params, f'Mean Relative Humidity {year1} (%)')
        Map.addLayer(humidity2, vis_params, f'Mean Relative Humidity {year2} (%)')

    except Exception as e:
        print(f"Error loading humidity layers: {e}")

# Function to create wind speed layers
def create_wind_speed_layers(year1, year2):
    def load_wind_speed(year):
        return ee.ImageCollection("ECMWF/ERA5/MONTHLY") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) \
            .mean() \
            .expression('sqrt(u*u + v*v)', {'u': 'u_component_of_wind_10m', 'v': 'v_component_of_wind_10m'})

    try:
        wind_speed1 = load_wind_speed(year1).clip(MP)
        wind_speed2 = load_wind_speed(year2).clip(MP)

        vis_params = {
            'min': 0,
            'max': 5,
            'palette': ['lightgreen', 'yellow', 'orange', 'red']
        }

        Map.addLayer(wind_speed1, vis_params, f'Mean Wind Speed {year1} (m/s)')
        Map.addLayer(wind_speed2, vis_params, f'Mean Wind Speed {year2} (m/s)')

    except Exception as e:
        print(f"Error loading wind speed layers: {e}")

# Function to create evaporation layers
def create_evaporation_layers(year1, year2):
    def load_evaporation(year):
        return ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .select('evaporation_from_vegetation_transpiration') \
            .sum() \
            .multiply(-1)  # Convert from negative to positive values

    try:
        evaporation1 = load_evaporation(year1).clip(MP)
        evaporation2 = load_evaporation(year2).clip(MP)

        vis_params = {
            'min': 0,
            'max': 1000,
            'palette': ['lightyellow', 'orange', 'red', 'purple']
        }

        Map.addLayer(evaporation1, vis_params, f'Annual Evaporation {year1} (mm)')
        Map.addLayer(evaporation2, vis_params, f'Annual Evaporation {year2} (mm)')

    except Exception as e:
        print(f"Error loading evaporation layers: {e}")

# Function to create temperature anomalies layers
def create_temperature_anomalies_layers(year1, year2):
    def load_temperature_anomalies(year):
        baseline = ee.ImageCollection("ECMWF/ERA5/MONTHLY") \
            .filterDate('1981-01-01', '2010-12-31') \
            .select('mean_2m_air_temperature') \
            .mean()
        
        target_year = ee.ImageCollection("ECMWF/ERA5/MONTHLY") \
            .filterDate(f"{year}-01-01", f"{year}-12-31") \
            .select('mean_2m_air_temperature') \
            .mean()
        
        return target_year.subtract(baseline)

    try:
        anomalies1 = load_temperature_anomalies(year1).clip(MP)
        anomalies2 = load_temperature_anomalies(year2).clip(MP)

        vis_params = {
            'min': -2,
            'max': 2,
            'palette': ['blue', 'white', 'red']
        }

        Map.addLayer(anomalies1, vis_params, f'Temperature Anomalies {year1} (°C)')
        Map.addLayer(anomalies2, vis_params, f'Temperature Anomalies {year2} (°C)')

    except Exception as e:
        print(f"Error loading temperature anomalies layers: {e}")

# Create dropdowns for year and climate factor selection
years = list(range(1979, 2025))  # Using 1979 as the start year to accommodate all datasets
year1_dropdown = widgets.Dropdown(options=years, value=2000, description='Year 1:')
year2_dropdown = widgets.Dropdown(options=years, value=2024, description='Year 2:')

climate_factors = ['Land Surface Temperature', 'Rainfall', 'Humidity', 'Wind Speed', 'Evaporation', 'Temperature Anomalies']
climate_factor_dropdown = widgets.Dropdown(options=climate_factors, value='Land Surface Temperature', description='Climate Factor:')

# Function to update layers based on selected climate factor and years
def update_layers(change):
    # Clear existing layers
    while len(Map.layers) > 1:
        Map.remove_layer(Map.layers[-1])
    
    factor = climate_factor_dropdown.value
    year1 = year1_dropdown.value
    year2 = year2_dropdown.value

    if factor == 'Land Surface Temperature':
        create_lst_layers(year1, year2)
    elif factor == 'Rainfall':
        create_rainfall_layers(year1, year2)
    elif factor == 'Humidity':
        create_humidity_layers(year1, year2)
    elif factor == 'Wind Speed':
        create_wind_speed_layers(year1, year2)
    elif factor == 'Evaporation':
        create_evaporation_layers(year1, year2)
    elif factor == 'Temperature Anomalies':
        create_temperature_anomalies_layers(year1, year2)
    
    Map.centerObject(MP, 6)

# Observe changes in all dropdowns
year1_dropdown.observe(update_layers, names='value')
year2_dropdown.observe(update_layers, names='value')
climate_factor_dropdown.observe(update_layers, names='value')

# Display the map and dropdowns
display(widgets.VBox([climate_factor_dropdown, year1_dropdown, year2_dropdown, Map]))

# Initial layer creation
update_layers(None)